### **Import necessary python modules**

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Global constants


In [ ]:
""" This module defines physical and mathematical constants for atmospheric modeling. Careful for the units! """

MAXNZ = 50 # maximum number of layers
R_Gas = 8.3145e7 # Universal gas constant erg/(K mol)
StefBoltz = 5.6704e-5 # Stefan-Boltzmann constant erg/(cm^2 s K^4)
Avogadro = 6.02214179e23 # Avogadro's number molecules/mol
k_Boltz = 1.38064852e-16 # Boltzmann constant erg/K
Pi = np.pi

# Implementation of read_voyager.f
This code snippet talks about the way the input file from voyager_inputs in the main eddysed code has been handled.


In [ ]:
"""The input file from voyager_inputs in the main eddysed code.
 reads the following inputs:
 Pressure 
 Temperature
 Gravity
 Effective Temperature
 Number of layers edges

 It calculates the number of layers , effective convective heat flux (chf), 
 molecular weight of atmosphere (assumed constant at 2.2 g/mol), and builds the T and P layers and sub layers along with the creation of altitude layers (Z) and its sub layers.


 and returns the following outputs:

gravity, T_eff, num_layer_edges, p_layer_edges, t_layer_edges, z_layer_edges, num_layers,  p_layers, t_layers, z_layers, chf



"""


def read_voyager(input_file):

    input_data = []
    with open(input_file, "r") as f:
        lines = f.readlines()
        T_eff = float(lines[0])
        gravity = float(lines[1])
        num_layer_edges = int(lines[2])
        num_layers = num_layer_edges - 1
        for line in lines[4:]:
            parts = line.strip().split()
            input_data.append([float(x.replace("E", "e")) for x in parts])
    input_array = np.array(input_data)
    p_layer_edges = input_array[:, 0]
    t_layer_edges = input_array[:, 1]

    r_atmos = 8.3145e7/2.2

    z_layer_edges = np.zeros(num_layer_edges)
    z_layers = np.zeros(num_layers)
    p_layers = np.zeros(num_layers)
    t_layers = np.zeros(num_layers)

    for i in range(num_layers-1, -1, -1):
        itop = i
        ibot = i + 1
        dlnp = np.log(p_layer_edges[ibot]/p_layer_edges[itop])
        p_layers[i] = (p_layer_edges[itop] + p_layer_edges[ibot]) / 2
        scale_height = r_atmos * t_layer_edges[ibot] / gravity
        dz_layer = scale_height * dlnp
        z_layer_edges[i] = z_layers[ibot-1] + dz_layer
        dtdz = (t_layer_edges[itop] - t_layer_edges[ibot]) / dz_layer
        dz_pmid = scale_height * np.log(p_layer_edges[ibot]/p_layers[i])
        z_layers[i] = z_layer_edges[ibot] + dz_pmid
        t_layers[i] = t_layer_edges[ibot] + dtdz * dz_pmid

    chf = StefBoltz * T_eff**4 * np.ones(num_layers)

    return gravity, T_eff, num_layer_edges, p_layer_edges, t_layer_edges, z_layer_edges, num_layers,  p_layers, t_layers, z_layers, chf

# Implementation of calc_qc.f

In [ ]:
def calc_qc(gas_name, f_sed_layer, rho_p, mw_cloud, q_below, supsat, w_convect, mixl, dz_layer, gravity, mw_atmos, mfp, visc, t_layer, p_layer, sig_layer):

    pvap = pvap_gas(gas_name, t_layer, p_layer)
    fs = supsat + 1
    rho_atmos = p_layer / (R_Gas/mw_cloud * t_layer)
    qvs = fs * pvap / (R_Gas/mw_atmos * t_layer) / rho_atmos

    # Layer is cloud free
    if q_below <= qvs:

        qt_layer = q_below
        qt_top = q_below
        qc_layer = 0.0
        rg_layer = 0.0
        reff_layer = 0.0
        ndz_layer = 0

    # Layer has cloud
    else:
        
        # range of mixing ratios to search (g/g)
        qhigh = q_below
        qlow = qhigh * 0.001

        # precision of advective diffusive solution (g/g)
        delta_q = q_below /1000

        # load paramaters into advdiff common block
        ad_qbelow = q_below
        ad_qvs = qvs
        ad_mixl = mixl
        ad_dz = dz_layer
        ad_fsed = f_sed_layer

        # Find total condensate mixing ratio at the top of the layer
        qt_top, status_q = find_root(advdiff, qlow, qhigh, delta_q)

        # Use trapezoid rule to calculate layer averages
        # -- should integrate exponential
        qt_layer = 0.5 * (q_below + qt_top)

        # Diagnose condensate mixing ratio
        qc_layer = max(0.0, qt_layer - qvs)

        # Find rw corresponding to w_convect using function vfall()

        # load parameters into vfall common block
        vf_grav = gravity
        vf_mw_atmos = mw_atmos
        vf_mfp = mfp
        vf_visc = visc
        vf_p = p_layer
        vf_t = t_layer
        vf_rhop = rho_p

        # range of particle radii to search (cm)
        rlow = 1e-10
        rhigh = 10

        # precision of terminal velocity solution (cm/s)
        delta_v = w_convect / 1000
        rw_layer, status_r = find_root(vfall, rlow, rhigh, delta_v)

        # geometric std dev of lognormal size distribution
        lnsig2 = 0.5*np.log( sig_layer )**2

        # Compute exponent in vfall = w_convect r^alpha
        # sigma floor  for the purpose of alpha calculation
        sig_alpha = max(1.1, sig_layer)
        
        # Bulk of precip at r > rw: exponent between rw and rw*sig
        if f_sed_layer > 1:
            alpha = np.log(vfall(rw_layer * sig_alpha) / w_convect) / np.log(sig_alpha)
        
        # Bulk of precip at r < rw: exponent between rw/sig and rw
        else:
            alpha = np.log(w_convect / vfall(rw_layer / sig_alpha)) / np.log(sig_alpha)
        
        # geometric mean radius of lognormal size distribution
        rg_layer = f_sed_layer**(1/alpha) * rw_layer * np.exp(-(alpha+6)*lnsig2)

        # droplet effective radius (cm)
        reff_layer = rg_layer * np.exp(5*lnsig2)

        # Column droplet number concentration (cm^-2)
        ndz_layer = 3.0 * rho_atmos * qc_layer * dz_layer / (4.0 * Pi * rho_p * rg_layer**3 * np.exp(-9 * lnsig2))

    return qc_layer, qt_layer, rg_layer, reff_layer, ndz_layer, qt_top, status_r, status_q

# Implementation of layer.f

In [ ]:
def layer(nsub_max, gas_name, kz_min, cloudf_min, mw_cloud, mw_atmos, f_sed, rho_p, supsat, sig_layer, t_layer, p_layer, t_top, t_bot, p_top, p_bot, chf_layer, gravity, q_below, report_status_r, report_status_q):

    # Set error return codes to zero
    status_r = 0
    status_q = 0

    # Number of levels of grid refinement used 
    nsub = 1

    # diameter of atmospheric molecule (cm) (Rosner, 2000)
    # (3.711e-8 for air, 3.798e-8 for N2, 2.827e-8 for H2)
    d_molecule = 2.827e-8

    # parameter in Lennard-Jones potential for viscosity (K) (Rosner, 2000)
    # (78.6 for air, 71.4 for N2, 59.7 for H2)
    eps_k = 59.7

    # specific gas constant for atmosphere (erg/(g K))
    r_atmos = R_Gas / mw_atmos

    # specific heat of atmosphere (erg/K/g)
    c_p = 7./2. * r_atmos

    # pressure thickness of layer
    dp_layer = p_bot - p_top
    dlnp = np.log( p_bot/p_top )

    # temperature gradient 
    dtdlnp = ( t_top - t_bot ) / dlnp
    lapse_ratio = ( t_bot - t_top ) / dlnp / ( 2./7.*t_layer )

    # atmospheric density (g/cm^3)
    rho_atmos = p_layer / ( r_atmos * t_layer )

    # atmospheric scale height (cm)
    scale_h = r_atmos * t_layer / gravity

    # convective mixing length scale (cm): no less than 1/10 scale height
    mixl = max( 0.1, lapse_ratio ) * scale_h

    # mixing length = scale height matches Lunine (1989) model
    mixl = scale_h

    # scale factor for eddy diffusion: 1/3 is baseline
    scalef_kz = 1./3.

    # vertical eddy diffusion coefficient (cm^2/s)
    # from Gierasch and Conrath (1985)
    kz = scalef_kz * scale_h * (mixl/scale_h)**(4/3) *( ( r_atmos*chf_layer) / ( rho_atmos*c_p ) )**(1/3)

    # no less than minimum value (for radiative regions)
    kz = max( kz, kz_min )

    # convective velocity scale (cm/s)
    w_convect = kz / mixl

    # cloud fractional coverage
    cloudf = cloudf_min + max( 0, min( 1, 1-lapse_ratio )) * ( 1 - cloudf_min )

    # atmospheric number density (molecules/cm^3)
    n_atmos = p_layer / ( k_Boltz*t_layer )

    # atmospheric mean free path (cm)
    mfp = 1. / ( np.sqrt(2.)*n_atmos*Pi*d_molecule**2 )

    # atmospheric viscosity (dyne s/cm^2)
    visc = 5./16.*np.sqrt( Pi*k_Boltz*t_layer*(mw_atmos/Avogadro)) / ( Pi*d_molecule**2 ) / ( 1.22 * ( t_layer / eps_k )**(-0.16) )

    # --------------------------------------------------------------------
    # Top of convergence layer
    
    converge = False
    while not converge:
        
        # Zero cumulative values
        qc_layer = 0
        qt_layer = 0
        ndz_layer = 0
        opd_layer = 0

        # total mixing ratio and pressure at the bottom of sub layer
        qt_bot_sub = q_below
        p_bot_sub = p_bot

        # Loop over sub layers
        dp_sub = dp_layer / nsub
        for isub in range(nsub):

            qt_below = qt_bot_sub
            p_top_sub = p_bot_sub - dp_sub
            dz_sub = scale_h * np.log( p_bot_sub / p_top_sub )
            p_sub = ( p_bot_sub + p_top_sub ) / 2
            t_sub = t_bot + dtdlnp * np.log( p_bot / p_sub )

            # Calculate condensate mixing ratio etc. for sub layer
            qc_sub, qt_sub, rg_sub, reff_sub, ndz_sub, qt_top, status_r, status_q = calc_qc(gas_name, f_sed, rho_p, mw_cloud, qt_below, supsat, w_convect, mixl, dz_sub, gravity, mw_atmos, mfp, visc, t_sub, p_sub, sig_layer)

            # Vertical sums
            qc_layer += qc_sub * dp_sub/gravity
            qt_layer += qt_sub * dp_sub/gravity
            ndz_layer += ndz_sub

            if reff_sub > 0:
                opd_layer += 1.5 * qc_sub * dp_sub / gravity / (rho_p * reff_sub)
            
            # Increment values at the bottom of the sub layer
            qt_bot_sub = qt_top
            p_bot_sub = p_top_sub
        
            # Check convergence on optical depth
            if nsub_max > 1:
                converge = True
            elif nsub == 1:
                opd_test = opd_layer
            elif (opd_layer > 0) or (nsub >= nsub_max):
                converge = True
            elif np.abs(1 - opd_test/opd_layer) < 0.01:
                converge = True
            else:
                opd_test = opd_layer
            
            nsub = nsub * 2
        
        # --------------------------------------------------------------------
        # Bottom of convergence layer

        # Report problems finding the root first time it happens
        if (status_r != 0) and report_status_r:
            print("layer():")
            print(f" find_root(vfall) status = {status_r:3d} for {gas_name} at p = {p_layer/1e6:10.2e}")
            print("There may be more instances not reported")
            print("")
            print(f"status_r = {status_r:3d}")
            report_status_r = False
        
        if (status_r != 0) and report_status_q:
            print("layer():")
            print(f" find_root(advdiff) status = {status_q:3d} for {gas_name} at p = {p_layer/1e6:10.2e}")
            print(" there may be more instances not reported")
            print()
            print(f"status_q = {status_q}")
            report_status_q = False

        # Update properties at the bottom of next layer
        q_below = qt_top

        # Get layer averages

        if opd_layer > 0:
            reff_layer = 1.5 * qc_layer / (rho_p * opd_layer)
            lnsig2 = 0.5 * np.log( sig_layer )**2
            rg_layer = reff_layer * np.exp(-5 * lnsig2)
        else:
            reff_layer = 0.0
            rg_layer = 0.0

        qc_layer = qc_layer * gravity/dp_layer
        qt_layer = qt_layer * gravity/dp_layer

    return qc_layer, qt_layer, rg_layer, reff_layer, ndz_layer, opd_layer, report_status_r, report_status_q

# Implementation of eddysed.f

In [7]:
def eddysed(gravity, T_eff, kz_min, cloudf_min, nsub_max, supsat, mw_atmos, do_virtual, num_layers, z_layers, z_layer_edges, p_layers, p_layer_edges, t_layers, t_layer_edges, chf, gas_name, gas_mmr, gas_mw, rho_p, sig, f_sed):
    
    # Initialization:
    cloud_base = 0
    report_status_r = True
    report_status_q = True

    # Calculate indices of bottom and top of domain
    if z[1] > z[0]:
        k_bottom = 0
        k_top = num_layers - 1
        increment = 1
    else:
        k_bottom = num_layers - 1
        k_top = 0
        increment = -1

    pass